## PASSIVE MODEL
Setup simulation of passive model

In [1]:
import os, sys
sys.path.append(os.path.split(sys.path[0])[0])

#Dependencies
import h5py
from neuron import h
import pandas as pd
import numpy as np

#Project Imports
import config.params as params
import config.paths as paths
from stylized_module.base.passive_model_soma_injection import *


geo_standard = pd.read_csv(paths.GEO_STANDARD,index_col='id')
h.tstop = params.PM_TSTOP
h.dt = params.PM_DT

Load data and determine boundaries

In [2]:
hf = h5py.File(paths.SIMULATED_DATA_FILE, 'r')
groundtruth_lfp = np.array(hf.get('data'))
hf.close()

maxIndx = np.argmax(np.absolute(groundtruth_lfp).max(axis=0))  # find maximum absolute value from averaged traces
maxTrace = -groundtruth_lfp[params.PM_START_IDX:,maxIndx]
x0_trace = groundtruth_lfp[params.PM_START_IDX:params.PM_START_IDX+params.PM_WINDOW_SIZE,:]
soma_injection = np.insert(maxTrace,0,0.)
soma_injection = np.asarray([s * params.PM_SCALING_FACTOR for s in soma_injection])

Create and run simulation

In [3]:
sim = Simulation(geo_standard,params.PM_ELECTRODE_POSITION,soma_injection)
sim.run()
t = sim.t()
t0 = t[:params.PM_WINDOW_SIZE]

Calculate PM_SCALING_FACTOR

In [4]:
scaling=(30-(sim.cells[0]._vrest))/(np.max(sim.cells[0].v())-(sim.cells[0]._vrest))
print(scaling)

1085.000000000031
